In [1]:
import os
import math
import json
import argparse
import random
from typing import Tuple, Dict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import mean_absolute_error, mean_squared_error
import statsmodels.api as sm


def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def safe_mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    if mask.sum() == 0:
        return float('nan')
    return float(np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0)

def mae(y_true, y_pred):
    return float(mean_absolute_error(y_true, y_pred))

def rmse(y_true, y_pred):
    return float(math.sqrt(mean_squared_error(y_true, y_pred)))


In [2]:
def generate_synthetic_multivariate(
    timesteps: int = 1200,
    n_series: int = 3,
    noise_scale: float = 0.1,
    seed: int = 42
) -> Tuple[np.ndarray, np.ndarray]:
    rng = np.random.RandomState(seed)
    t = np.arange(timesteps)
    data = np.zeros((timesteps, n_series))

    global_trend = 0.0005 * (t ** 1.2)

    for s in range(n_series):
        amp1 = 1.0 + 0.3 * s
        amp2 = 0.6 + 0.2 * (s % 2)
        phase = s * 0.5

        season1 = amp1 * np.sin(2 * np.pi * t / 24 + phase)
        season2 = amp2 * np.cos(2 * np.pi * t / (24 * 7) + phase * 0.3)
        local_trend = (0.0002 * (t ** 1.1)) * (1 + 0.1 * s)
        regime = np.where(t > timesteps * 0.6, 0.5 * np.sin(0.002 * (t - timesteps * 0.6)), 0.0)
        noise = rng.normal(scale=noise_scale * (1 + 0.2 * s), size=timesteps)

        data[:, s] = 2.0 * global_trend + local_trend + season1 + season2 + regime + noise

    data = (data - data.mean(axis=0)) / (data.std(axis=0) + 1e-8)
    return t, data


In [3]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data: np.ndarray, lookback: int, horizon: int):
        assert data.ndim == 2, "data must be shape (timesteps, n_series)"
        self.data = data.astype(np.float32)
        self.lookback = int(lookback)
        self.horizon = int(horizon)
        self.T, self.C = self.data.shape

    def __len__(self):
        return max(0, self.T - self.lookback - self.horizon + 1)

    def __getitem__(self, idx):
        x = self.data[idx: idx + self.lookback]
        y = self.data[idx + self.lookback: idx + self.lookback + self.horizon]
        return x.T.copy(), y.T.copy()


In [4]:
class NBeatsBlock(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, theta_dim: int):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim, theta_dim)
        )
    def forward(self, x):
        return self.net(x)

class NBeatsModel(nn.Module):
    def __init__(self, input_length: int, horizon: int, n_series: int, hidden_units: int = 256, stacks: int = 2, blocks_per_stack: int = 3, theta_trend: int = 4, theta_season: int = 8, season_period: float = 24.0, device: torch.device = torch.device('cpu')):
        super().__init__()
        self.input_length = input_length
        self.horizon = horizon
        self.n_series = n_series
        self.device = device
        self.theta_trend = int(theta_trend)
        self.theta_season = int(theta_season)
        self.season_period = float(season_period)
        self.theta_per_series = 2 * self.theta_trend + 4 * self.theta_season
        self.blocks = nn.ModuleList()
        self.num_blocks = stacks * blocks_per_stack
        input_dim = n_series * input_length
        for _ in range(self.num_blocks):
            theta_dim = n_series * self.theta_per_series
            self.blocks.append(NBeatsBlock(input_dim, hidden_units, theta_dim))
        self.final_fc = nn.Linear(self.num_blocks * n_series * horizon, n_series * horizon)
        self.register_buffer('trend_back_v', torch.tensor(np.vander(np.linspace(-1, 0, input_length), N=self.theta_trend, increasing=True), dtype=torch.float32))
        self.register_buffer('trend_fore_v', torch.tensor(np.vander(np.linspace(0, 1, horizon), N=self.theta_trend, increasing=True), dtype=torch.float32))
        sincos_back = []
        sincos_fore = []
        b_time = np.arange(input_length)
        f_time = np.arange(horizon)
        for k in range(1, self.theta_season + 1):
            omega = 2 * math.pi * k / self.season_period
            sincos_back.append(np.sin(omega * b_time))
            sincos_back.append(np.cos(omega * b_time))
            sincos_fore.append(np.sin(omega * f_time))
            sincos_fore.append(np.cos(omega * f_time))
        self.register_buffer('season_back_b', torch.tensor(np.stack(sincos_back, axis=0), dtype=torch.float32))
        self.register_buffer('season_fore_b', torch.tensor(np.stack(sincos_fore, axis=0), dtype=torch.float32))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        batch = x.shape[0]
        device = x.device
        inp = x.reshape(batch, -1)
        forecasts_blocks = []
        for block in self.blocks:
            theta = block(inp)
            theta = theta.view(batch, self.n_series, self.theta_per_series)
            per_series_f = []
            for s in range(self.n_series):
                the_s = theta[:, s, :]
                t_len = self.theta_trend
                s_len = self.theta_season
                idx = 0
                trend_theta = the_s[:, idx: idx + 2 * t_len]
                idx += 2 * t_len
                season_theta = the_s[:, idx: idx + 4 * s_len]
                tb = trend_theta[:, :t_len]
                tf = trend_theta[:, t_len:]
                b_trend = torch.matmul(tb, self.trend_back_v.to(device).T)
                f_trend = torch.matmul(tf, self.trend_fore_v.to(device).T)
                bs = season_theta[:, :s_len]
                bc = season_theta[:, s_len: 2 * s_len]
                fs = season_theta[:, 2 * s_len: 3 * s_len]
                fc = season_theta[:, 3 * s_len: 4 * s_len]
                coeff_back = torch.stack([v for pair in zip(bs.split(1, dim=1), bc.split(1, dim=1)) for v in pair], dim=1).squeeze(-1)
                coeff_fore = torch.stack([v for pair in zip(fs.split(1, dim=1), fc.split(1, dim=1)) for v in pair], dim=1).squeeze(-1)
                b_seas = torch.matmul(coeff_back, self.season_back_b.to(device))
                f_seas = torch.matmul(coeff_fore, self.season_fore_b.to(device))
                f = f_trend + f_seas
                per_series_f.append(f.unsqueeze(1))
            f_block = torch.cat(per_series_f, dim=1)
            forecasts_blocks.append(f_block)
        concat = torch.cat([fb.reshape(batch, -1) for fb in forecasts_blocks], dim=1)
        out = self.final_fc(concat)
        out = out.view(batch, self.n_series, self.horizon)
        return out


In [5]:
def sarima_forecast(series: np.ndarray, steps: int = 24) -> np.ndarray:
    try:
        model = sm.tsa.SARIMAX(series, order=(1, 1, 1), seasonal_order=(1, 1, 1, 24),
                               enforce_stationarity=False, enforce_invertibility=False)
        res = model.fit(disp=False, maxiter=50)
        f = res.forecast(steps)
        return np.asarray(f)
    except Exception:
        return np.ones(steps) * series[-1]

class LSTMForecast(nn.Module):
    def __init__(self, input_len: int, horizon: int, n_series: int, hidden: int = 64, layers: int = 1):
        super().__init__()
        self.lstm = nn.LSTM(input_size=n_series, hidden_size=hidden, num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden, n_series * horizon)
        self.horizon = horizon
        self.n_series = n_series
    def forward(self, x):
        x = x.permute(0, 2, 1)
        out, _ = self.lstm(x)
        last = out[:, -1, :]
        out = self.fc(last)
        out = out.view(x.size(0), self.n_series, self.horizon)
        return out


In [6]:
def train_one_epoch(model, loader, optimizer, criterion, device, clip_grad=None):
    model.train()
    total_loss = 0.0
    for x_batch, y_batch in loader:
        x = torch.tensor(x_batch, dtype=torch.float32, device=device)
        y = torch.tensor(y_batch, dtype=torch.float32, device=device)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        if clip_grad:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)
        optimizer.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(loader.dataset)

def eval_model(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    preds = []
    trues = []
    with torch.no_grad():
        for x_batch, y_batch in loader:
            x = torch.tensor(x_batch, dtype=torch.float32, device=device)
            y = torch.tensor(y_batch, dtype=torch.float32, device=device)
            pred = model(x)
            loss = criterion(pred, y)
            total_loss += loss.item() * x.size(0)
            preds.append(pred.cpu().numpy())
            trues.append(y.cpu().numpy())
    preds = np.concatenate(preds, axis=0)
    trues = np.concatenate(trues, axis=0)
    return total_loss / len(loader.dataset), preds, trues

def compute_and_print_metrics(preds: np.ndarray, trues: np.ndarray, label: str = "Model"):
    results = {}
    y_true = trues.reshape(-1)
    y_pred = preds.reshape(-1)
    results['global_mae'] = mae(y_true, y_pred)
    results['global_rmse'] = rmse(y_true, y_pred)
    results['global_mape'] = safe_mape(y_true, y_pred)
    print(f"\n=== {label} GLOBAL METRICS ===")
    print(f"MAE:  {results['global_mae']:.6f}")
    print(f"RMSE: {results['global_rmse']:.6f}")
    print(f"MAPE: {results['global_mape']:.4f}%")
    C = preds.shape[1]
    per_series = {}
    print(f"\nPer-series metrics ({C} series):")
    for s in range(C):
        yt = trues[:, s, :].reshape(-1)
        yp = preds[:, s, :].reshape(-1)
        per_series[s] = {
            'mae': mae(yt, yp),
            'rmse': rmse(yt, yp),
            'mape': safe_mape(yt, yp)
        }
        print(f" Series {s}: MAE={per_series[s]['mae']:.6f}, RMSE={per_series[s]['rmse']:.6f}, MAPE={per_series[s]['mape']:.4f}%")
    H = preds.shape[2]
    per_horizon = {}
    print(f"\nPer-horizon MAE (first 8 shown):")
    per_h_mae = np.mean(np.abs(preds - trues), axis=(0,1))
    per_h_mape = np.mean(np.abs((preds - trues) / (trues + 1e-8)), axis=(0,1)) * 100.0
    for h in range(H):
        per_horizon[h] = {'mae': float(per_h_mae[h]), 'mape': float(per_h_mape[h])}
        if h < 8:
            print(f"  step +{h+1:2d}: MAE={per_h_mae[h]:.6f}, MAPE={per_h_mape[h]:.4f}%")
    results['per_series'] = per_series
    results['per_horizon'] = per_horizon
    return results


In [7]:
def seasonal_naive_forecast(history: np.ndarray, steps: int, season_period: int = 24) -> np.ndarray:
    if len(history) < season_period:
        return np.ones(steps) * history[-1]
    last_season = history[-season_period:]
    reps = int(np.ceil(steps / season_period))
    tiled = np.tile(last_season, reps)[:steps]
    return tiled

def compute_relative_improvement(base_metric: float, model_metric: float) -> float:
    if base_metric == 0:
        return float('nan')
    return (base_metric - model_metric) / base_metric * 100.0

def bootstrap_prediction_intervals(preds: np.ndarray, trues: np.ndarray, n_bootstrap: int = 500, alpha: float = 0.05, random_state: int = 42):
    rng = np.random.RandomState(random_state)
    N, C, H = preds.shape
    boot_means = np.zeros((n_bootstrap, C, H), dtype=np.float32)
    for b in range(n_bootstrap):
        idx = rng.randint(0, N, size=N)
        sample_preds = preds[idx]
        boot_means[b] = sample_preds.mean(axis=0)
    lower = np.percentile(boot_means, 100 * (alpha / 2.0), axis=0)
    upper = np.percentile(boot_means, 100 * (1 - alpha / 2.0), axis=0)
    return lower, upper


In [8]:
def run_experiment_notebook(args):
    set_seed(args.seed)
    device = torch.device("cuda" if torch.cuda.is_available() and not args.force_cpu else "cpu")
    os.makedirs(args.artifacts_dir, exist_ok=True)
    t, data = generate_synthetic_multivariate(timesteps=args.timesteps, n_series=args.n_series, noise_scale=args.noise, seed=args.seed)
    T, C = data.shape
    train_cut = int(T * args.train_ratio)
    val_cut = int(T * (args.train_ratio + args.val_ratio))
    train_data = data[:train_cut]
    val_data = data[train_cut:val_cut]
    test_data = data[val_cut:]
    train_ds = TimeSeriesDataset(np.vstack([train_data, val_data]), args.lookback, args.horizon)
    test_ds = TimeSeriesDataset(test_data, args.lookback, args.horizon)
    train_loader = DataLoader(train_ds, batch_size=args.batch_size, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=args.batch_size, shuffle=False)
    model = NBeatsModel(input_length=args.lookback, horizon=args.horizon, n_series=C,
                        hidden_units=args.hidden, stacks=args.stacks, blocks_per_stack=args.blocks,
                        theta_trend=args.theta_trend, theta_season=args.theta_season,
                        season_period=args.season_period, device=device).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=3)
    criterion = nn.MSELoss()
    best_val = float('inf')
    best_state = None
    wait = 0
    for epoch in range(1, args.epochs + 1):
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device, clip_grad=args.grad_clip)
        val_loss, _, _ = eval_model(model, test_loader, criterion, device)
        scheduler.step(val_loss)
        print(f"Epoch {epoch:02d}/{args.epochs} - train_loss={train_loss:.6f} - val_loss={val_loss:.6f} - lr={optimizer.param_groups[0]['lr']:.2e}")
        if val_loss + 1e-9 < best_val:
            best_val = val_loss
            best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            wait = 0
        else:
            wait += 1
            if wait >= args.patience:
                print("Early stopping: no improvement.")
                break
    if best_state is not None:
        model.load_state_dict(best_state)
    _, preds, trues = eval_model(model, test_loader, criterion, device)
    preds = preds.reshape(-1, C, args.horizon)
    trues = trues.reshape(-1, C, args.horizon)
    metrics_nbeats = compute_and_print_metrics(preds, trues, label="N-BEATS (final)")
    torch.save(model.state_dict(), os.path.join(args.artifacts_dir, "nbeats_final.pth"))
    with open(os.path.join(args.artifacts_dir, "metrics_nbeats.json"), "w") as f:
        json.dump(metrics_nbeats, f, indent=2)
    preds_flat = preds.reshape(-1, C * args.horizon)
    trues_flat = trues.reshape(-1, C * args.horizon)
    pd.DataFrame(preds_flat).to_csv(os.path.join(args.artifacts_dir, "preds_nbeats.csv"), index=False)
    pd.DataFrame(trues_flat).to_csv(os.path.join(args.artifacts_dir, "trues_nbeats.csv"), index=False)
    lstm = LSTMForecast(args.lookback, args.horizon, C, hidden=args.lstm_hidden, layers=args.lstm_layers).to(device)
    opt_l = torch.optim.Adam(lstm.parameters(), lr=args.lr)
    for e in range(args.lstm_epochs):
        _ = train_one_epoch(lstm, train_loader, opt_l, criterion, device, clip_grad=args.grad_clip)
    _, p_l, t_l = eval_model(lstm, test_loader, criterion, device)
    p_l = p_l.reshape(-1, C, args.horizon)
    t_l = t_l.reshape(-1, C, args.horizon)
    metrics_lstm = compute_and_print_metrics(p_l, t_l, label="LSTM (baseline)")
    torch.save(lstm.state_dict(), os.path.join(args.artifacts_dir, "lstm_baseline.pth"))
    with open(os.path.join(args.artifacts_dir, "metrics_lstm.json"), "w") as f:
        json.dump(metrics_lstm, f, indent=2)
    sarima_preds = []
    sarima_trues = []
    for i in range(len(test_data) - args.lookback - args.horizon + 1):
        history = np.vstack([train_data, val_data, test_data[:i + args.lookback]])
        f = np.zeros((C, args.horizon))
        for s in range(C):
            f[s, :] = sarima_forecast(history[:, s], steps=args.horizon)
        sarima_preds.append(f[np.newaxis, ...])
        y_true = test_data[i + args.lookback: i + args.lookback + args.horizon].T
        sarima_trues.append(y_true[np.newaxis, ...])
    if sarima_preds:
        sarima_preds = np.concatenate(sarima_preds, axis=0)
        sarima_trues = np.concatenate(sarima_trues, axis=0)
        metrics_sarima = compute_and_print_metrics(sarima_preds, sarima_trues, label="SARIMA (baseline)")
        with open(os.path.join(args.artifacts_dir, "metrics_sarima.json"), "w") as f:
            json.dump(metrics_sarima, f, indent=2)
    # Seasonal naive baseline
    seasonal_preds = []
    seasonal_trues = []
    for i in range(len(test_data) - args.lookback - args.horizon + 1):
        history = np.vstack([train_data, val_data, test_data[:i + args.lookback]])
        f = np.zeros((C, args.horizon))
        for s in range(C):
            f[s, :] = seasonal_naive_forecast(history[:, s], steps=args.horizon, season_period=int(args.season_period))
        seasonal_preds.append(f[np.newaxis, ...])
        y_true = test_data[i + args.lookback: i + args.lookback + args.horizon].T
        seasonal_trues.append(y_true[np.newaxis, ...])
    if seasonal_preds:
        seasonal_preds = np.concatenate(seasonal_preds, axis=0)
        seasonal_trues = np.concatenate(seasonal_trues, axis=0)
        metrics_seasonal = compute_and_print_metrics(seasonal_preds, seasonal_trues, label="Seasonal Naive (baseline)")
        nbeats_mape = metrics_nbeats['global_mape']
        naive_mape = metrics_seasonal['global_mape']
        mape_impr = compute_relative_improvement(naive_mape, nbeats_mape)
        nbeats_mae = metrics_nbeats['global_mae']
        naive_mae = metrics_seasonal['global_mae']
        mae_impr = compute_relative_improvement(naive_mae, nbeats_mae)
        print(f"\nRelative improvement over Seasonal Naive:\n  MAPE improvement: {mape_impr:.2f}%\n  MAE improvement:  {mae_impr:.2f}%")
    n_boot = args.bootstrap_n if args.bootstrap_n > 0 else 0
    if n_boot > 0:
        print(f"\nComputing bootstrap prediction intervals with {n_boot} samples...")
        lower_ci, upper_ci = bootstrap_prediction_intervals(preds, trues, n_bootstrap=n_boot, alpha=0.05, random_state=args.seed)
        for s in range(C):
            df_ci = pd.DataFrame({'horizon_step': np.arange(1, args.horizon + 1), 'lower': lower_ci[s, :], 'upper': upper_ci[s, :]})
            df_ci.to_csv(os.path.join(args.artifacts_dir, f"nbeats_bootstrap_ci_series{s}.csv"), index=False)
        print("Bootstrap CI CSVs saved to artifacts/")
        s = 0
        sample_idx = 0
        input_window = test_data[sample_idx: sample_idx + args.lookback, s]
        true_future = test_data[sample_idx + args.lookback: sample_idx + args.lookback + args.horizon, s]
        pred_mean = preds[sample_idx, s, :]
        lower = lower_ci[s, :]
        upper = upper_ci[s, :]
        timeline = np.arange(sample_idx, sample_idx + args.lookback + args.horizon)
        plt.figure(figsize=(10,4))
        plt.plot(timeline[:args.lookback], input_window, label='input')
        plt.plot(timeline[args.lookback:], true_future, label='true')
        plt.plot(timeline[args.lookback:], pred_mean, label='nbeats_pred', lw=2)
        plt.fill_between(timeline[args.lookback:], lower, upper, color='gray', alpha=0.25, label='95% CI')
        plt.legend()
        plt.title('Forecast + 95% CI (series 0)')
        plt.tight_layout()
        plt.savefig(os.path.join(args.artifacts_dir, 'example_forecast_with_ci_series0.png'), dpi=150)
        plt.close()
    summary = {
        'nbeats_metrics': metrics_nbeats,
        'lstm_metrics': metrics_lstm,
        'artifact_dir': os.path.abspath(args.artifacts_dir)
    }
    with open(os.path.join(args.artifacts_dir, 'summary.json'), 'w') as f:
        json.dump(summary, f, indent=2)
    print('\nAll artifacts saved to:', os.path.abspath(args.artifacts_dir))

    return metrics_nbeats


In [ ]:
class Args:
    pass

def default_args():
    a = Args()
    a.timesteps = 1200
    a.n_series = 3
    a.lookback = 72
    a.horizon = 24
    a.batch_size = 64
    a.lr = 1e-3
    a.epochs = 60
    a.patience = 8
    a.seed = 42
    a.train_ratio = 0.7
    a.val_ratio = 0.15
    a.noise = 0.1
    a.hidden = 256
    a.stacks = 2
    a.blocks = 3
    a.theta_trend = 4
    a.theta_season = 8
    a.season_period = 24.0
    a.grad_clip = 1.0
    a.force_cpu = False
    a.artifacts_dir = 'artifacts'
    a.weight_decay = 1e-6
    a.lstm_hidden = 128
    a.lstm_layers = 1
    a.lstm_epochs = 20
    a.bootstrap_n = 200
    return a

  if __name__ == '__main__':
    args = default_args()
    run_experiment_notebook(args)


/tmp/ipython-input-763811516.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x_batch, dtype=torch.float32, device=device)
/tmp/ipython-input-763811516.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y_batch, dtype=torch.float32, device=device)
/tmp/ipython-input-763811516.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x_batch, dtype=torch.float32, device=device)
/tmp/ipython-input-763811516.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceT

Epoch 01/60 - train_loss=0.200790 - val_loss=0.075901 - lr=1.00e-03
Epoch 02/60 - train_loss=0.039698 - val_loss=0.095244 - lr=1.00e-03
Epoch 03/60 - train_loss=0.016516 - val_loss=0.017617 - lr=1.00e-03
Epoch 04/60 - train_loss=0.007048 - val_loss=0.010835 - lr=1.00e-03
Epoch 05/60 - train_loss=0.005071 - val_loss=0.007476 - lr=1.00e-03
Epoch 06/60 - train_loss=0.004496 - val_loss=0.006162 - lr=1.00e-03
Epoch 07/60 - train_loss=0.004138 - val_loss=0.006318 - lr=1.00e-03
Epoch 08/60 - train_loss=0.004156 - val_loss=0.006092 - lr=1.00e-03
Epoch 09/60 - train_loss=0.003937 - val_loss=0.008300 - lr=1.00e-03
Epoch 10/60 - train_loss=0.003904 - val_loss=0.006142 - lr=1.00e-03
Epoch 11/60 - train_loss=0.003864 - val_loss=0.006019 - lr=1.00e-03
Epoch 12/60 - train_loss=0.004312 - val_loss=0.006094 - lr=1.00e-03
Epoch 13/60 - train_loss=0.004127 - val_loss=0.006923 - lr=1.00e-03
Epoch 14/60 - train_loss=0.003950 - val_loss=0.006868 - lr=1.00e-03
Epoch 15/60 - train_loss=0.004321 - val_loss=0.0

/tmp/ipython-input-763811516.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x_batch, dtype=torch.float32, device=device)
/tmp/ipython-input-763811516.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y_batch, dtype=torch.float32, device=device)
/tmp/ipython-input-763811516.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x_batch, dtype=torch.float32, device=device)
/tmp/ipython-input-763811516.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceT


=== LSTM (baseline) GLOBAL METRICS ===
MAE:  0.145507
RMSE: 0.177216
MAPE: 10.6727%

Per-series metrics (3 series):
 Series 0: MAE=0.133502, RMSE=0.162624, MAPE=8.8281%
 Series 1: MAE=0.133254, RMSE=0.163303, MAPE=10.0067%
 Series 2: MAE=0.169763, RMSE=0.202738, MAPE=13.1833%

Per-horizon MAE (first 8 shown):
  step + 1: MAE=0.106192, MAPE=8.9073%
  step + 2: MAE=0.096433, MAPE=8.3370%
  step + 3: MAE=0.107479, MAPE=9.2479%
  step + 4: MAE=0.116584, MAPE=9.0629%
  step + 5: MAE=0.131455, MAPE=9.8756%
  step + 6: MAE=0.125630, MAPE=9.2983%
  step + 7: MAE=0.141506, MAPE=10.1081%
  step + 8: MAE=0.135908, MAPE=9.5705%


/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.12/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
